In [4]:
import sys
import os
import requests
import csv
from config import get_settings
from schemas import (
    ProviderEnum,
    ByteMeQueryParams,
    ApiRequestHeaders,
)
import requests
import logging
import time
import json
import hmac
import hashlib

In [5]:
settings = get_settings()
payload = {
  "city": "München",
  "houseNumber": "5",
  "plz": "80666",
  "street": "Musterstraße",
  "wantsFiber": False
}
payload_as_json = json.dumps(
    payload, ensure_ascii=False, separators=(",", ":")
)
current_timestamp = int(time.time())
json_and_timestamp = f"{current_timestamp}:{payload_as_json}"

hashed_payload = hmac.new(
    key=settings.PINGPERFECT_SIGNATURE_SECRET.encode(),
    msg=json_and_timestamp.encode(),
    digestmod=hashlib.sha256,
).hexdigest()

headers = {
    "Content-Type": "application/json",
    "X-Client-Id": settings.PINGPERFECT_CLIENT_ID,
    "X-Signature": hashed_payload,
    "X-Timestamp": str(current_timestamp),
}


In [6]:
response = requests.post(
    url=settings.PINGPERFECT_URL,
    headers=headers,
    data=payload_as_json,
)

In [14]:
a = response.json()

In [16]:
type(a)

list

In [4]:
settings = get_settings()
settings.PINGPERFECT_CLIENT_ID

'BDBCA328'

In [ ]:
settings = get_settings()
query_params = {
    "street": "Musterstraße",
    "houseNumber": "5",
    "city": "München",
    "plz": "80634",
}

query_params = ByteMeQueryParams(**query_params).model_dump(by_alias=True)
headers = ApiRequestHeaders(
    content_type="text/xml; charset=utf-8", x_api_key=settings.BYTEME_API_KEY
).model_dump(by_alias=True)
response = requests.get(
    settings.BYTEME_BASE_URL,
    headers=headers,
    params=query_params,  # Add query parameters here
)

In [ ]:
response.text

In [ ]:
reader = csv.DictReader(response.text.splitlines())

In [ ]:
raw_products = [row for row in csv.DictReader(response.text.splitlines())]

In [ ]:
raw_products

In [ ]:
len(raw_products)

In [ ]:
# remove duplicates from list with dicts
unique_products = {frozenset(item.items()): item for item in raw_products}.values()

In [ ]:
len([dict(t) for t in {tuple(d.items()) for d in raw_products}])